# Generate Promo Codes (12‑char) with Letter/Digit Rules → CSV / Excel

This notebook generates **unique** promo codes with configurable rules and writes them to
a CSV file (streaming) and an Excel file. It uses cryptographically secure randomness.

**Quick start:**
1. Run the first two cells to install/import deps and set your rules.
2. Run the main generation cell.
3. Files will be created in the working directory.

### What "certain letter requirements" can I enforce?
- Minimum counts for letters and digits
- Allowed character set (e.g., A–Z + 0–9, excluding ambiguous characters)
- Must include at least one char from specific sets (e.g., must contain one of AEIOU)
- Optional prefix/suffix
- Forbid patterns via regular expressions (e.g., no three repeats, no specific substrings)

You can tweak the CONFIG block below.

%%
(Optional) If running on a very minimal environment, uncomment to ensure pandas/openpyxl are present.\
%pip install -q pandas openpyxl

## Configuration

In [5]:
!python --version

Python 3.11.13


In [13]:
import csv
import os
import re
import sqlite3
from secrets import choice as secure_choice
from string import ascii_uppercase, digits
import pandas as pd

In [14]:
OUTPUT_CSV = "promo_codes.csv"
OUTPUT_XLSX = "promo_codes.xlsx"
TOTAL_CODES = 100 # 1200000
CODE_LENGTH = 12

# Define your allowed character set. By default: A–Z and 0–9 excluding easily confused chars.
AMBIGUOUS = set("0O1I5S") # you can set to empty set() to allow everything
ALPHABET = [c for c in ascii_uppercase + digits if c not in AMBIGUOUS]

# Rule knobs — set to None to disable a rule
MIN_LETTERS = 4 # require at least 4 letters A–Z in each code
MIN_DIGITS = 4 # require at least 4 digits 0–9 in each code (note ALPHABET may exclude some digits)
REQUIRE_ONE_OF = set("AEUX") # require at least one of these letters (set() to disable)
PREFIX = "" # e.g., "ACME"
SUFFIX = "" # e.g., "25"

# Forbid patterns (provide regexes as raw strings). Examples included; comment out to disable.
FORBIDDEN_REGEXES = [
r"(.)\1{2,}", # no 3+ of the same char in a row
r"AAAA|EEEE|IIII|OOOO|UUUU", # forbid long vowel runs
]

# Uniqueness backend: 'memory' (fast, high RAM) or 'sqlite' (lower RAM, slower, very robust)
UNIQUENESS_BACKEND = "sqlite" # choose "memory" or "sqlite"
SQLITE_DB_PATH = "promo_codes_unique.db" # used only if backend is sqlite
SQLITE_TABLE = "codes"

# Chunking — how many to buffer before writing to Excel (CSV writes line-by-line)
EXCEL_CHUNK_SIZE = 100000

## Helper functions

In [15]:
_letter_re = re.compile(r"[A-Z]")
_digit_re = re.compile(r"[0-9]")
_forbidden_res = [re.compile(p) for p in FORBIDDEN_REGEXES]


def passes_rules(code: str) -> bool:
    """Return True if the code meets all configured rules."""
    # Prefix/suffix
    if PREFIX and not code.startswith(PREFIX):
        return False
    if SUFFIX and not code.endswith(SUFFIX):
        return False

    # Core slice to validate (exclude prefix/suffix regions from counts if provided)
    core = code

    # Counts
    if MIN_LETTERS is not None:
        if len(_letter_re.findall(core)) < MIN_LETTERS:
            return False
        if MIN_DIGITS is not None:
            if len(_digit_re.findall(core)) < MIN_DIGITS:
                return False

    # Must include at least one from REQUIRE_ONE_OF
    if REQUIRE_ONE_OF:
        if not any(ch in REQUIRE_ONE_OF for ch in core):
            return False

    # Forbidden patterns
    for rx in _forbidden_res:
        if rx.search(core):
            return False

    return True


def random_code() -> str:
    body_len = CODE_LENGTH - len(PREFIX) - len(SUFFIX)
    if body_len <= 0:
        raise ValueError("CODE_LENGTH too small for given PREFIX/SUFFIX")
    body = ''.join(secure_choice(ALPHABET) for _ in range(body_len))
    return f"{PREFIX}{body}{SUFFIX}"


class UniqueSink:
    """Track uniqueness either in memory or via SQLite."""
    def __init__(self, backend: str = "memory"):
        self.backend = backend
        if backend == "memory":
            self._seen = set()
        elif backend == "sqlite":
            self._conn = sqlite3.connect(SQLITE_DB_PATH)
            cur = self._conn.cursor()
            cur.execute(f"CREATE TABLE IF NOT EXISTS {SQLITE_TABLE} (code TEXT PRIMARY KEY)")
            self._conn.commit()
        else:
            raise ValueError("backend must be 'memory' or 'sqlite'")

    def add_if_new(self, code: str) -> bool:
        if self.backend == "memory":
            if code in self._seen:
                return False
            self._seen.add(code)
            return True
        else:
            try:
                cur = self._conn.cursor()
                cur.execute(f"INSERT INTO {SQLITE_TABLE}(code) VALUES (?)", (code,))
                self._conn.commit()
                return True
            except sqlite3.IntegrityError:
                return False

    def close(self):
        if self.backend == "sqlite":
            self._conn.close()

## Main generation loop

In [17]:
# Remove old output files if they exist
for p in [OUTPUT_CSV, OUTPUT_XLSX, SQLITE_DB_PATH]:
    if os.path.exists(p):
        os.remove(p)

unique = UniqueSink(UNIQUENESS_BACKEND)

written = 0
rows_for_excel = []

with open(OUTPUT_CSV, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["code"])

    while written < TOTAL_CODES:
        code = random_code()
        if not passes_rules(code):
            continue
        if not unique.add_if_new(code):
            continue

        writer.writerow([code])
        written += 1

        rows_for_excel.append(code)
        if len(rows_for_excel) >= EXCEL_CHUNK_SIZE:
            df = pd.DataFrame({"code": rows_for_excel})
            mode = "w" if written <= EXCEL_CHUNK_SIZE else "a"
            with pd.ExcelWriter(
                OUTPUT_XLSX,
                engine="openpyxl",
                mode=mode,
                if_sheet_exists=("overlay" if mode == "a" else None),
            ) as xw:
                startrow = 0
                if mode == "a":
                    try:
                        existing = pd.read_excel(OUTPUT_XLSX, sheet_name=0, usecols=[0])
                        startrow = len(existing) + 1
                    except Exception:
                        startrow = 0
                df.to_excel(xw, index=False, sheet_name="codes", startrow=startrow)
            rows_for_excel.clear()

# Handle remaining rows
if rows_for_excel:
    df = pd.DataFrame({"code": rows_for_excel})
    mode = "w" if not os.path.exists(OUTPUT_XLSX) else "a"
    with pd.ExcelWriter(
        OUTPUT_XLSX,
        engine="openpyxl",
        mode=mode,
        if_sheet_exists=("overlay" if mode == "a" else None),
    ) as xw:
        startrow = 0
        if mode == "a":
            try:
                existing = pd.read_excel(OUTPUT_XLSX, sheet_name=0, usecols=[0])
                startrow = len(existing) + 1
            except Exception:
                startrow = 0
        df.to_excel(xw, index=False, sheet_name="codes", startrow=startrow)

unique.close()
print(f"Done. Wrote {written:,} unique codes to {OUTPUT_CSV} and {OUTPUT_XLSX}.")

Done. Wrote 100 unique codes to promo_codes.csv and promo_codes.xlsx.
